# All Models Test

Load the right file and rull all

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.svm import SVR
import keras
import keras.backend as kb
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.metrics import r2_score

In [8]:
def printScores(reg, X_test, y_test):
    regName = reg.__class__().__str__()[:reg.__class__().__str__().index("(")]
    
    print("\nR2 score : {} = {}".format(regName, reg.score(X_test, y_test)))
    print("RMSE     : {} = {}".format(regName, sqrt(mean_squared_error(y_test, reg.predict(X_test)))))


In [9]:
def run_models(X, y, nn_input):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=42)
    
    lReg = LinearRegression()
    lReg.fit(X_train, y_train)
    printScores(lReg, X_test, y_test)
#     print("\nR2 score : Linear Regressor =",lReg.score(X_test, y_test))

#     from sklearn.preprocessing import PolynomialFeatures
#     pReg = PolynomialFeatures(degree = 4)
#     X_poly = pReg.fit_transform(X_train)
#     lin_reg = LinearRegression()
#     lin_reg.fit(X_poly, y_train)

    brReg = BayesianRidge()
    brReg.fit(X_train, y_train)
    printScores(brReg, X_test, y_test)
#     print("\nR2 score : BayesianRidge =", brReg.score(X_test, y_test))
    
    dtReg = DecisionTreeRegressor()
#     print(dtReg)
    dtReg.fit(X_train, y_train)
    printScores(dtReg, X_test, y_test)
#     print("\nR2 score : DecisionTreeRegressor =",dtReg.score(X_test, y_test))
    
    rfReg = RandomForestRegressor(n_estimators = 10, random_state = 0)
    rfReg.fit(X_train, y_train)
    printScores(rfReg, X_test, y_test)
#     print("\nR2 score : RandomForestRegressor =",rfReg.score(X_test, y_test))

    svReg = SVR(kernel = 'rbf', gamma='scale')
    svReg.fit(X_train, y_train)
    printScores(svReg, X_test, y_test)
#     print("\nR2 score : RandomForestRegressor =",rfReg.score(X_test, y_test))
    
    # NN
    model = keras.Sequential([
        keras.layers.Dense(134, activation=tf.nn.relu, input_shape=[nn_input]),
        keras.layers.Dense(134, activation=tf.nn.relu),
        keras.layers.Dense(134, activation=tf.nn.relu),
        keras.layers.Dense(1)
      ])

    optimizer = tf.keras.optimizers.RMSprop(0.0099)
#     model = tf.keras.models.load_model('./ipl_model_tf')
    model.compile(loss="mean_squared_error",optimizer=optimizer)
    model.fit(X_train,y_train,epochs=500, validation_split=0.2, shuffle=True, verbose=0)
    model.save('./ipl_model_tf',save_format='tf')
    regName = 'Neural Network'
    print("\nR2 score : {} = {}".format(regName, r2_score(model.predict(X_test), y_test)))
    print("RMSE     : {} = {}".format(regName, sqrt(mean_squared_error(y_test, model.predict(X_test)))))
    
    # Visualising the Decision Tree Regression Results 
    plt.figure(figsize=(15,10))
    X_grid = np.arange(0, len(X))
    #.reshape(-1, 1)
    plt.scatter(X_grid, y, color = 'black')
    plt.plot(X_grid, lReg.predict(X), color = 'red')
#     plt.plot(X_grid, dtReg.predict(X), color = 'green')
    plt.plot(X_grid, rfReg.predict(X), color = 'blue')
    plt.plot(X_grid, brReg.predict(X), color = 'pink')
    plt.plot(X_grid, svReg.predict(X), color = 'yellow')
    plt.plot(X_grid, model.predict(X), color = 'cyan', marker='x', lineWidth=0)
    plt.title('All Models')
    plt.xlabel('index number')
    plt.ylabel('Revenue')
    plt.show()


In [12]:
def do_traning(data=None, filename=None):
    if(data is None and filename is None):
        raise Exception("Atleast provide data or filename")
    
    if(data is None):
        data = pd.read_csv('./dt/feature_engg_1.csv', header=0)
    
    df = data.copy()
#     df.sample(5)
    print("Data has shape ", df.shape)
    
    y = df['target']
    print('y_shape is ', y.shape)
    X = df.drop('target', axis=1)
    print('x_shape is ',X.shape)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=42)
    
    run_models(X, y, X.shape[1])
    
    

In [11]:
# saving the model in tensorflow format
# model.save('./MyModel_tf',save_format='tf')

# loading the saved model
# loaded_model = tf.keras.models.load_model('./MyModel_tf')

# retraining the model
# loaded_model.fit(X_train,y_train,epochs=500)